Load the MNIST dataset and split it into training and test sets.

In [ ]:
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

Preprocess the data by normalizing the pixel values.

In [ ]:
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

Add Gaussian noise to the images.

In [ ]:
import numpy as np
noise_factor = 0.5
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
x_train_noisy = np.clip(x_train_noisy, 0.0, 1.0)

Build a simple autoencoder model using Keras.

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
input_img = Input(shape=(28, 28))
h = Dense(64, activation='relu')(input_img)
encoded = Dense(32, activation='relu')(h)
decoded = Dense(64, activation='sigmoid')(encoded)
output_img = Dense(28, activation='sigmoid')(decoded)
autoencoder = Model(input_img, output_img)

Train the autoencoder on the noisy images.

In [ ]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.fit(x_train_noisy, x_train, epochs=50, batch_size=256, shuffle=True, validation_data=(x_test, x_test))

Denoise the test images using the trained autoencoder.

In [ ]:
denoised_images = autoencoder.predict(x_test_noisy)

Evaluate the performance of the autoencoder using Mean Squared Error.

In [ ]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(x_test.flatten(), denoised_images.flatten())

Train a classifier on the denoised images.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()
classifier.fit(denoised_images.reshape(-1, 28*28), y_test)

Evaluate the classifier's performance on the denoised images.

In [ ]:
classifier_score = classifier.score(denoised_images.reshape(-1, 28*28), y_test)

Create a composite model combining the autoencoder and classifier.

In [ ]:
composite_model = Model(inputs=input_img, outputs=classifier.predict(denoised_images.reshape(-1, 28*28)))

Make predictions using the composite model.

In [ ]:
predictions = composite_model.predict(x_test_noisy)